In [1]:
# Read all QALD-8 Queries
# For each question
    # Extract all URIs
    # Find sameAs links to DE, FR, ES dbpedia
    # If links exist in same language for all URIs then;
        # generate sparql for all languages
        # write the sparql like: "query_<language-abbr>": { "sparql": <sparql> }
# Store the new QALD File

In [26]:
# Load QALD8 questions

# import urllib library
from urllib.request import urlopen
  
# import json
import json
# store the URL in url as 
# parameter for urlopen
url = "https://raw.githubusercontent.com/ag-sc/QALD/master/7/data/qald-7-train-multilingual.json"
  
# store the response of URL
response = urlopen(url)
  
# storing the JSON response 
# from url in data
data_json = json.loads(response.read())
  
# print the json response
print(data_json['dataset']['id'])

qald-7-train-multilingual


In [27]:
import time
from SPARQLWrapper import SPARQLWrapper, JSON
from string import Template

sparql = SPARQLWrapper("http://dbpedia.org/sparql")
# Choosing to fetch only one link
sparql_template = 'SELECT ?l WHERE { <$uri> owl:sameAs ?l . FILTER(regex(str(?l), "http://$lang.dbpedia.org/" )) }'
sparql_template2 = 'SELECT ?l WHERE { <$uri> owl:sameAs ?l . FILTER(regex(str(?l), "http://$lang.dbpedia.org/" )) } LIMIT 1'
def fetch_links(uri, lang):
    links_set = set()
    # form the sparql
    query = Template(sparql_template).substitute(uri=uri, lang=lang)
    # query
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()

    #print('Result size:',len(results["results"]["bindings"]))

    for result in results["results"]["bindings"]:
        links_set.add(result['l']['value'])
    
    time.sleep(0.1)
    return links_set

def fetch_single_link(uri, lang):
    single_link = None
    # form the sparql
    query = Template(sparql_template2).substitute(uri=uri, lang=lang)
    # query
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()

    #print('Result size:',len(results["results"]["bindings"]))

    for result in results["results"]["bindings"]:
        single_link = result['l']['value']
    
    time.sleep(1)
    return single_link

# function to check and return links for URIs in a given language
def check_links(uri_list, lang):
    missing_links = False
    # For each uri look for language specific links
    link_map = {}
    for uri in uri_list:
        links_set = fetch_links(uri, lang)
        if len(links_set) == 0:
            missing_links = True
        link_map[uri] = links_set
    return (missing_links, link_map)

In [28]:
# Test Block
# print(check_links({'http://dbpedia.org/resource/Barack_Obama','http://dbpedia.org/resource/Donald_Trump'},'de'))

In [22]:
endpoint_dict = { 'de': 'http://de.dbpedia.org/sparql', 'es': 'https://es.dbpedia.org/sparql', 'fr': 'http://fr.dbpedia.org/sparql'}

In [23]:
# Convert SPARQL
def convert_sparql(sparql_str, prefix_tuples, uris, lang):
    link_map = {}
    # fetch the links for tuples
    for entry in prefix_tuples:
        id = entry[0]
        uri = entry[1]
        single_link = fetch_single_link(uri, lang)
        if not single_link:
            return None
        link_map[id] = single_link
    # fetch the links for uris
    for uri in uris:
        single_link = fetch_single_link(uri, lang)
        if not single_link:
            return None
        link_map[uri] = single_link
    # create sparql
    # use these three lines to do the replacement
    rep = dict((re.escape(k), v) for k, v in link_map.iteritems()) 
    #Python 3 renamed dict.iteritems to dict.items so use rep.items() for latest versions
    pattern = re.compile("|".join(rep.keys()))
    sparql_str = pattern.sub(lambda m: rep[re.escape(m.group(0))], sparql_str)
    # return the sparql
    return sparql
def get_all_sparql(sparql_str, prefix_tuples, uris):
    sparql_map = {}
    # for each language fetch the sparql queries
    for lang in endpoint_dict.keys():
        sparql_map[key] = convert_sparql(sparql_str, prefix_tuples, uris, lang)
    # return the queries
    return sparql_map

In [24]:
# Testing replacement


In [25]:
import re
prefix_map = {}
uris = set()
for question in data_json['questions']:
    sparql_uris = set()
    sparql_str = question['query']['sparql']    
   # extract all prefixes to make a map
    prefix_tuples=re.findall('PREFIX\s+([a-zA-Z0-9]+):\s+<(.*?)>',sparql_str, re.IGNORECASE)
    for entry in prefix_tuples:
        prefix_map[entry[0]] = entry[1]
    #print(prefix_map)
    # extract all prefix mentions
    prefix_mentions = re.findall('{?[\s\t\n\r]+([\w\d]+):([^<\s]+)',sparql_str, re.IGNORECASE)
    #print(prefix_mentions)
    # join the local name with prefix and store it in uri set
    
    for entry in prefix_mentions:
        sparql_uris.add(prefix_map[entry[0]]+entry[1])
    # print(uris)
    # extract all direct uri mentions
    extra_uris = re.findall('<(.*?)>',sparql_str, re.IGNORECASE)
    sparql_uris.update(extra_uris)
    # Generate and save language specific SPARQL queries
    sparql_map = get_all_sparql(sparql_str, prefix_tuples, extra_uris)
    print(sparql_map)
    uris.update(sparql_uris)
    break
print('Total', len(uris) , 'unique URIs found.')

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1108)>